## COLLECTE DES DONNEES (API ADEME) NEUFS ET EXISTANTS

In [11]:
import os
import time
import requests
import pandas as pd
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import datetime as dt
from requests.exceptions import ChunkedEncodingError, ConnectionError
from urllib3.exceptions import ProtocolError

## CONFIGURATION

In [12]:
DATA_DIR = "../data"
os.makedirs(DATA_DIR, exist_ok=True)

DATASETS = {
    "existants":"https://data.ademe.fr/data-fair/api/v1/datasets/dpe03existant/lines",
    "neufs":"https://data.ademe.fr/data-fair/api/v1/datasets/dpe02neuf/lines",
}
SCHEMA_COLS = {}
for label, url in DATASETS.items():
    try:
        r = requests.get(url.replace("/lines", "/schema"))
        r.raise_for_status()
        schema_cols = [f["key"] for f in r.json()]
        SCHEMA_COLS[label] = schema_cols
        print(f"[INFO] Schéma chargé pour {label} : {len(schema_cols)} colonnes.")
    except Exception as e:
        print(f"[WARN] Impossible de charger le schéma pour {label}: {e}")
        SCHEMA_COLS[label] = None


DEPT_CODE = "71"  # c'est le champ à modifier pour choisir le département visé
CP_PATTERN = f"{DEPT_CODE}*" # sert à formatter le Code département pour la requête API, 69* → CP commençant par 69
YEARS = range(2021, 2026)   # période test
PAGE_SIZE = 1200

OUT = {
    "existants": os.path.join(DATA_DIR,f"donnees_dpe_existants_{DEPT_CODE}.csv"),
    "neufs":     os.path.join(DATA_DIR,f"donnees_dpe_neufs_{DEPT_CODE}.csv"),
}


[INFO] Schéma chargé pour existants : 229 colonnes.


KeyboardInterrupt: 

## Session

In [ ]:
session = requests.Session()
retries = Retry(
    total=5,
    connect=3,
    read=3,
    backoff_factor=0.6,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"],
)
session.mount("https://", HTTPAdapter(max_retries=retries))

## FONCTIONS UTILITAIRES

In [ ]:
def fetch_first_page(base_url: str, year: int):
    """Récupère la première page pour l'année donnée."""
    borne1 = f"{year}-01-01"
    borne2 = f"{year}-12-31"
    params = {
        "size": PAGE_SIZE,
        "sort":"date_reception_dpe", # tri par date croissante
        "q": CP_PATTERN,
        "q_fields": "code_postal_ban",
        "qs": f"date_reception_dpe:[{borne1} TO {borne2}]",
    }
    r = session.get(base_url, params=params, timeout=60)
    r.raise_for_status()
    return r.json()

def fetch_next_page(next_url: str, max_retries: int = 3, delay: float = 5.0):
    """Récupère la page suivante à partir du champ 'next' avec tolérance aux coupures réseau."""
    for attempt in range(1, max_retries + 1):
        try:
            r = session.get(next_url, timeout=120)
            r.raise_for_status()
            return r.json()

        except (ChunkedEncodingError, ConnectionError, ProtocolError) as e:
            print(f"[WARN] Connexion interrompue (tentative {attempt}/{max_retries}) : {e}")
            if attempt < max_retries:
                time.sleep(delay * attempt)  # backoff progressif
                continue
            else:
                print(f"[ERROR] Échec permanent après {max_retries} tentatives. Page ignorée.")
                return {"results": [], "next": None}

        except requests.exceptions.RequestException as e:
            print(f"[ERROR] Erreur HTTP inattendue : {e}")
            time.sleep(delay)
    return {"results": [], "next": None}

def append_to_csv(df: pd.DataFrame, path: str, header_manager: dict):
    """Écrit ou ajoute au CSV en respectant le schéma initial."""
    if header_manager.get("columns") is None:
        header_manager["columns"] = list(df.columns)
        df = df.reindex(columns=header_manager["columns"])
        df.to_csv(path, index=False, mode="w", header=True)
    else:
        df = df.reindex(columns=header_manager["columns"])
        df.to_csv(path, index=False, mode="a", header=False)

## COLLECTE PRINCIPALE

In [ ]:
def collect_dpe(label: str, base_url: str, out_csv: str):
    header_manager = {"columns": None}
    total_rows_written = 0
    print(f"\n=== COLLECTE [{label.upper()}] ===")

    # --- Vérification existence du fichier ---
    if os.path.exists(out_csv):
        try:
            existing_df = pd.read_csv(out_csv, nrows=1)
            header_manager["columns"] = list(existing_df.columns)
            print(f"[INFO] Fichier existant détecté : les nouvelles données seront ajoutées à la suite ({out_csv}).")
        except Exception as e:
            print(f"[WARN] Impossible de lire le fichier existant ({e}), il sera recréé.")

    for year in YEARS:
        print(f"\n--- Année {year} ---")
        start_time = time.time()
        page = 1
        js = fetch_first_page(base_url, year)
        next_url = js.get("next")

        while True:
            results = js.get("results", [])
            if not results:
                print(f"[INFO] Aucune donnée pour {year}, page {page}")
                break

            df_page = pd.DataFrame(results)
            # --- Harmoniser les colonnes selon le schéma ADEME ---
            schema_cols = SCHEMA_COLS.get(label)
            if schema_cols:
                 df_page = df_page.reindex(columns=schema_cols)
            append_to_csv(df_page, out_csv, header_manager)
            total_rows_written += len(df_page)

            # --- Estimation du temps total après la première page ---
            if page == 1:
                elapsed = time.time() - start_time  # start_time défini avant la boucle année
                if js.get("total"):
                    estimated_total_time = (js["total"] / PAGE_SIZE) * elapsed
                    print(f"[INFO] Estimation de durée pour {year}: {dt.timedelta(seconds=int(estimated_total_time))}")

            print(f"Page {page:>3} | lignes: {len(df_page):>4} | total cumulé: {total_rows_written:,}")

            if not next_url:
                break  # plus de pages

            # Pause douce pour éviter le throttling
            time.sleep(0.5)

            # Page suivante
            js = fetch_next_page(next_url)
            next_url = js.get("next")
            page += 1

    print(f"\n✅ Terminé [{label}] : {out_csv} | {total_rows_written:,} lignes totales.\n")

## EXECUTION

In [ ]:
#neufs
collect_dpe("neufs",DATASETS["neufs"],OUT["neufs"])
print(f"\n🎯 Collecte neufs ({DEPT_CODE}) terminée")


=== COLLECTE [NEUFS] ===

--- Année 2021 ---
[INFO] Estimation de durée pour 2021: 0:00:00
Page   1 | lignes:  277 | total cumulé: 277

--- Année 2022 ---
[INFO] Estimation de durée pour 2022: 0:00:04
Page   1 | lignes:  896 | total cumulé: 1,173

--- Année 2023 ---
[INFO] Estimation de durée pour 2023: 0:00:06
Page   1 | lignes: 1146 | total cumulé: 2,319

--- Année 2024 ---
[INFO] Estimation de durée pour 2024: 0:00:04
Page   1 | lignes:  956 | total cumulé: 3,275

--- Année 2025 ---
[INFO] Estimation de durée pour 2025: 0:00:00
Page   1 | lignes:  347 | total cumulé: 3,622

✅ Terminé [neufs] : ../data\donnees_dpe_neufs_71.csv | 3,622 lignes totales.


🎯 Collecte neufs (71) terminée


In [ ]:
#existants
collect_dpe("existants",DATASETS["existants"],OUT["existants"])

print(f"\n🎯 Collecte existants ({DEPT_CODE}) terminée")


=== COLLECTE [EXISTANTS] ===

--- Année 2021 ---
[INFO] Estimation de durée pour 2021: 0:00:56
Page   1 | lignes: 1200 | total cumulé: 1,200
Page   2 | lignes: 1200 | total cumulé: 2,400
Page   3 | lignes: 1200 | total cumulé: 3,600
Page   4 | lignes: 1200 | total cumulé: 4,800
Page   5 | lignes: 1200 | total cumulé: 6,000
Page   6 | lignes:  814 | total cumulé: 6,814

--- Année 2022 ---
[INFO] Estimation de durée pour 2022: 0:03:50
Page   1 | lignes: 1200 | total cumulé: 8,014
Page   2 | lignes: 1200 | total cumulé: 9,214
Page   3 | lignes: 1200 | total cumulé: 10,414
Page   4 | lignes: 1200 | total cumulé: 11,614
Page   5 | lignes: 1200 | total cumulé: 12,814
Page   6 | lignes: 1200 | total cumulé: 14,014
Page   7 | lignes: 1200 | total cumulé: 15,214
Page   8 | lignes: 1200 | total cumulé: 16,414
Page   9 | lignes: 1200 | total cumulé: 17,614
Page  10 | lignes: 1200 | total cumulé: 18,814
Page  11 | lignes: 1200 | total cumulé: 20,014
Page  12 | lignes: 1200 | total cumulé: 21,214


In [ ]:
# Vérif nb lignes + vérif ANNEE

df_exist = pd.read_csv(f"../data/donnees_dpe_existants_{DEPT_CODE}.csv")
df_neuf = pd.read_csv(f"../data/donnees_dpe_neufs_{DEPT_CODE}.csv")

print("Existants :", df_exist.shape)
print("Neufs :", df_neuf.shape)

# Vérifier les années couvertes
print("\nAnnées existants :", df_exist["date_reception_dpe"].str[:4].value_counts().sort_index())
print("\nAnnées neufs :", df_neuf["date_reception_dpe"].str[:4].value_counts().sort_index())


C:\Users\ychen\AppData\Local\Temp\ipykernel_38628\2197104736.py:3: DtypeWarning: Columns (7,26,51,213) have mixed types. Specify dtype option on import or set low_memory=False.
  df_exist = pd.read_csv(f"../data/donnees_dpe_existants_{DEPT_CODE}.csv")


Existants : (85435, 229)
Neufs : (3622, 209)

Années existants : date_reception_dpe
2021     6814
2022    19884
2023    19047
2024    23703
2025    15987
Name: count, dtype: int64

Années neufs : date_reception_dpe
2021     277
2022     896
2023    1146
2024     956
2025     347
Name: count, dtype: int64


## Relance sur ANNEE, en cas de plantage pour dpe existant

In [14]:
# Années à relancer
YEARS = [2025]  # ou plusieurs : [2023, 2025]

# Nettoyer les lignes déjà présentes dans le CSV, pour ces années
path = f"../data/donnees_dpe_existants_{DEPT_CODE}.csv"
df = pd.read_csv(path)

print("Avant :", len(df))

# Convertir les années en chaînes et filtrer dynamiquement
years_str = [str(y) for y in YEARS]
df = df[~df["date_reception_dpe"].astype(str).str[:4].isin(years_str)]

print("Après suppression années", YEARS, ":", len(df))

# Sauvegarde du fichier nettoyé
df.to_csv(path, index=False)
print("✅ Fichier nettoyé, prêt pour re-collecte", YEARS)

# Exécution relance
collect_dpe("existants", DATASETS["existants"], OUT["existants"])


C:\Users\ychen\AppData\Local\Temp\ipykernel_38628\1104308851.py:6: DtypeWarning: Columns (7,26,51,213) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


Avant : 75483
Après suppression années [2025] : 69448
✅ Fichier nettoyé, prêt pour re-collecte [2025]

=== COLLECTE [EXISTANTS] ===
[INFO] Fichier existant détecté : les nouvelles données seront ajoutées à la suite (../data\donnees_dpe_existants_71.csv).

--- Année 2025 ---
[INFO] Estimation de durée pour 2025: 0:02:53
Page   1 | lignes: 1200 | total cumulé: 1,200
Page   2 | lignes: 1200 | total cumulé: 2,400
Page   3 | lignes: 1200 | total cumulé: 3,600
Page   4 | lignes: 1200 | total cumulé: 4,800
Page   5 | lignes: 1200 | total cumulé: 6,000
Page   6 | lignes: 1200 | total cumulé: 7,200
Page   7 | lignes: 1200 | total cumulé: 8,400
Page   8 | lignes: 1200 | total cumulé: 9,600
Page   9 | lignes: 1200 | total cumulé: 10,800
Page  10 | lignes: 1200 | total cumulé: 12,000
Page  11 | lignes: 1200 | total cumulé: 13,200
Page  12 | lignes: 1200 | total cumulé: 14,400
Page  13 | lignes: 1200 | total cumulé: 15,600
Page  14 | lignes:  387 | total cumulé: 15,987

✅ Terminé [existants] : ../d

In [15]:
path = f"../data/donnees_dpe_existants_{DEPT_CODE}.csv"
df = pd.read_csv(path)
print("Total lignes finale : ",len(df))

C:\Users\ychen\AppData\Local\Temp\ipykernel_38628\3833362665.py:2: DtypeWarning: Columns (7,26,51,213) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


Total lignes finale :  85435


## Test Size Requêtes

In [ ]:
#test size requetes

import time
import requests

BASE_URL = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe03existant/lines"
params_template = {
    "q": "69*",
    "q_fields": "code_postal_ban",
    "qs": "date_reception_dpe:[2022-01-01 TO 2022-12-31]"
}

sizes = [500, 1000, 2000, 5000, 10000, 11000]  # tailles à tester

for size in sizes:
    params = dict(params_template)
    params["size"] = size

    print(f"\n--- Test size={size} ---")
    start = time.time()
    r = requests.get(BASE_URL, params=params, timeout=120)
    duration = time.time() - start

    if r.status_code != 200:
        print(f"Erreur {r.status_code} : {r.text[:300]}")
        continue

    js = r.json()
    n_results = len(js.get("results", []))
    total = js.get("total", "N/A")

    print(f"Durée : {duration:.2f}s | Lignes retournées : {n_results} | Total annoncé : {total}")


--- Test size=500 ---


KeyboardInterrupt: 

In [ ]:
#test size requetes

import time
import requests

BASE_URL = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe03existant/lines"
params_template = {
    "q": "69*",
    "q_fields": "code_postal_ban",
    "qs": "date_reception_dpe:[2022-01-01 TO 2022-12-31]"
}

sizes = [1000, 1100, 1200, 1300, 1400,1500, 1600, 1700, 1800, 1900, 2000]  # tailles à tester

for size in sizes:
    params = dict(params_template)
    params["size"] = size

    print(f"\n--- Test size={size} ---")
    start = time.time()
    r = requests.get(BASE_URL, params=params, timeout=120)
    duration = time.time() - start

    if r.status_code != 200:
        print(f"Erreur {r.status_code} : {r.text[:300]}")
        continue

    js = r.json()
    n_results = len(js.get("results", []))
    total = js.get("total", "N/A")

    print(f"Durée : {duration:.2f}s | Lignes retournées : {n_results} | Total annoncé : {total}")
